<a href="https://colab.research.google.com/github/Anirookie/Anirookie/blob/main/travel_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Anirookie/Anirookie.git

Cloning into 'Anirookie'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 977.69 KiB | 10.18 MiB/s, done.
Resolving deltas: 100% (6/6), done.


# Data

In [2]:
import pandas as pd

sample_data = {
    "Text": ["We're planning a trip to Paris next summer!",
             "Thinking of visiting Rome in the fall.",
             "Considering a beach vacation in Hawaii next winter.",
             "Planning a ski trip to the Alps in January.",
             "Dreaming of exploring Tokyo next spring.",
             "Planning a business trip to New York in July.",
             "Looking forward to a Caribbean cruise in December.",
             "Hoping to visit London in the springtime.",
             "Considering a trip to Barcelona this autumn.",
             "Dreaming of a safari in Africa next summer.",
             "Exploring the historic streets of Istanbul in autumn.",
             "Heading to Moscow for a winter adventure.",
             "Planning a visit to the Great Wall of China next spring.",
             "Enjoying the cherry blossoms in Kyoto in April.",
             "Venturing to the jungles of Amazon next summer.",
             "Discovering the ancient ruins of Athens in autumn.",
             "Sailing the waters of the Mediterranean in summer.",
             "Trekking through the Himalayas in winter.",
             "Admiring the beauty of Niagara Falls in spring.",
             "Cruising along the coast of Alaska in summer.",
             "Embarking on a cultural journey to Paris in spring.",
             "Planning a road trip through Tuscany in summer.",
             "Hiking in the Rocky Mountains in autumn.",
             "Visiting the Vatican City in winter.",
             "Experiencing the magic of Disney World in summer.",
             "Attending the Carnival in Rio de Janeiro in February.",
             "Exploring the ancient temples of Angkor Wat in spring.",
             "Skiing in the Swiss Alps in winter.",
             "Indulging in the cuisine of New Orleans in autumn.",
             "Relaxing on the beaches of Maldives in summer.",
             "Camping under the stars in Yosemite National Park in fall.",
             "Attending Oktoberfest in Munich in September.",
             "Exploring the fjords of Norway in summer.",
             "Visiting the pyramids of Giza in winter.",
             "Taking a hot air balloon ride in Cappadocia in spring.",
             "Enjoying the tulip fields of Holland in April.",
             "Diving in the Great Barrier Reef in summer.",
             "Exploring the temples of Kyoto in autumn.",
             "Road tripping along the coast of California in summer.",
             "Visiting the Louvre Museum in Paris in winter.",
             "Cycling through the countryside of Provence in spring.",
             "Attending a music festival in Coachella Valley in April.",
             "Observing the Northern Lights in Iceland in winter.",
             "Exploring the historic streets of Prague in autumn.",
             "Visiting the Taj Mahal in India in spring.",
             "Cruising along the Danube River in summer.",
             "Hiking the Inca Trail to Machu Picchu in autumn.",
             "Experiencing the beauty of Santorini in Greece in summer."],
    "Destination": ["Paris", "Rome", "Hawaii", "Alps", "Tokyo", "New York", "Caribbean", "London", "Barcelona", "Africa",
                    "Istanbul", "Moscow", "China", "Kyoto", "Amazon", "Athens", "Mediterranean", "Himalayas", "Niagara Falls", "Alaska",
                    "Paris", "Tuscany", "Rocky Mountains", "Vatican City", "Disney World", "Rio de Janeiro", "Angkor Wat", "Swiss Alps", "New Orleans", "Maldives",
                    "Yosemite National Park", "Munich", "Norway", "Giza", "Cappadocia", "Holland", "Great Barrier Reef", "Kyoto", "California", "Paris", "Provence",
                    "Coachella Valley", "Iceland", "Prague", "Taj Mahal", "Danube River", "Machu Picchu", "Santorini"],
    "Time Period": ["Summer", "Fall", "Winter", "January", "Spring", "July", "December", "Spring", "Autumn", "Summer",
                    "Autumn", "Winter", "Spring", "April", "Summer", "Autumn", "Summer", "Winter", "Spring", "Summer",
                    "Spring", "Summer", "Autumn", "Winter", "Summer", "February", "Spring", "Winter", "Autumn", "Summer",
                    "Fall", "September", "Summer", "Winter", "Spring", "April", "Summer", "Autumn", "Summer", "Winter",
                    "Autumn", "April", "Winter", "Autumn", "Spring", "Summer", "Autumn", "Summer"],
    "Demand Forecast": ["High", "Medium", "High", "Medium", "High", "Medium", "High", "Medium", "High", "Low",
                        "Medium", "Medium", "High", "High", "Low", "Medium", "High", "Low", "High", "Medium",
                        "Medium", "Low", "Medium", "Medium", "High", "Low", "Medium", "High", "Low", "Medium",
                        "Medium", "High", "Medium", "Low", "High", "High", "Low", "Medium", "High", "Medium",
                        "High", "Low", "High", "Medium", "Low", "High", "Medium", "Low"]
}

df_sample = pd.DataFrame(sample_data)


# Dataset Construct


In [3]:
df_sample.head()

,Text,Destination,Time Period,Demand Forecast
0,We're planning a trip to Paris next summer!,Paris,Summer,High
1,Thinking of visiting Rome in the fall.,Rome,Fall,Medium
2,Considering a beach vacation in Hawaii next wi...,Hawaii,Winter,High
3,Planning a ski trip to the Alps in January.,Alps,January,Medium
4,Dreaming of exploring Tokyo next spring.,Tokyo,Spring,High


In [4]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
# Encode categorical variables
label_encoders = {}
for column in ['Destination', 'Time Period', 'Demand Forecast']:
    label_encoders[column] = LabelEncoder()
    df_sample[column] = label_encoders[column].fit_transform(df_sample[column])

In [6]:
# Split dataset into features and labels
X = df_sample[['Destination', 'Time Period']]
y = df_sample['Demand Forecast']

In [7]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
class TravelDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_texts, features, labels):
        self.tokenized_texts = tokenized_texts
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        input_ids = self.tokenized_texts['input_ids'][idx]
        attention_mask = self.tokenized_texts['attention_mask'][idx]
        label = self.labels.iloc[idx]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [9]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize dataset
# Tokenize dataset
tokenized_texts = tokenizer(list(df_sample['Text']), padding=True, truncation=True, return_tensors='pt')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
TravelDataset(tokenized_texts,features, labels).__getitem__(0)

NameError: name 'features' is not defined

In [10]:
!pip install datasets --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [11]:
from datasets import Dataset

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
hf_dataset = Dataset.from_dict(tokenized_texts)

# Upload the dataset to the Hub
hf_dataset.push_to_hub('Ani8Face/travel_log3')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ani8Face/travel_log3/commit/05df2e5b6e6651c5dd1cafd701b572623b9cf41d', commit_message='Upload dataset', commit_description='', oid='05df2e5b6e6651c5dd1cafd701b572623b9cf41d', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
!pip install wandb --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.6/281.6 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [13]:
import wandb

wandb.init(project="travel_logger",name='training_run')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Training

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import DistilBertForSequenceClassification, AdamW
from tqdm import tqdm

In [15]:
train_dataset = TravelDataset(tokenized_texts, X_train, y_train)
test_dataset = TravelDataset(tokenized_texts, X_test, y_test)

In [16]:
# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [17]:
# Initialize the pre-trained model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [19]:
for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    # Log loss to wandb
    wandb.log({"Epoch": epoch+1, "Train Loss": avg_loss})

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss}")

Epoch 1/3: 100%|██████████| 19/19 [00:02<00:00,  7.81it/s]


Epoch 1/3, Average Loss: 1.1227211575759084


Epoch 2/3: 100%|██████████| 19/19 [00:00<00:00, 26.79it/s]


Epoch 2/3, Average Loss: 1.0750391859757273


Epoch 3/3: 100%|██████████| 19/19 [00:00<00:00, 25.92it/s]

Epoch 3/3, Average Loss: 0.9857951402664185


## Validation track

In [20]:
from sklearn.model_selection import train_test_split

# Split dataset into train and validation sets

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create data loaders for train and validation sets
trainV_dataset = TravelDataset(tokenized_texts,X_train, y_train)
train_dataloader = DataLoader(trainV_dataset, batch_size=2, shuffle=True)

val_dataset = TravelDataset(tokenized_texts,X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

In [21]:


# Training loop
for epoch in range(num_epochs):
    total_train_loss = 0
    total_val_loss = 0

    # Training
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} (Train)"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)

    # Validation
    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} (Validation)"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)

    # Log training and validation loss to wandb
    wandb.log({"Epoch": epoch+1, "Train Loss": avg_train_loss, "Val Loss": avg_val_loss})

    # Print progress
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")


Epoch 1/3 (Validation): 100%|██████████| 4/4 [00:00<00:00, 142.35it/s]


Epoch 1/3, Train Loss: 1.0773826400438944, Val Loss: 1.2996747195720673


Epoch 2/3 (Validation): 100%|██████████| 4/4 [00:00<00:00, 147.93it/s]


Epoch 2/3, Train Loss: 0.9687555392583211, Val Loss: 1.277435064315796


Epoch 3/3 (Validation): 100%|██████████| 4/4 [00:00<00:00, 146.33it/s]

Epoch 3/3, Train Loss: 0.8476359089215596, Val Loss: 1.4275154918432236


### Eval & Prediction

In [22]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += torch.sum(predictions == labels).item()
        total += labels.size(0)

accuracy = correct / total * 100
# Log accuracy to wandb
wandb.log({"Test Accuracy": accuracy})

print(f"Accuracy: {accuracy:.2f}%")

Evaluating: 100%|██████████| 5/5 [00:00<00:00, 77.19it/s]

Accuracy: 50.00%


In [23]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluation
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions.extend(torch.argmax(outputs.logits, dim=1).tolist())
        true_labels.extend(labels.tolist())

# Compute accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print(classification_report(true_labels, predictions))

Evaluating: 100%|██████████| 5/5 [00:00<00:00, 139.16it/s]

Accuracy: 50.00%
              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.00      0.00      0.00         1
           2       0.60      0.75      0.67         4

    accuracy                           0.50        10
   macro avg       0.37      0.38      0.37        10
weighted avg       0.49      0.50      0.49        10



In [24]:
print(predictions)

[1, 0, 2, 2, 2, 2, 0, 0, 2, 0]


In [26]:
# import torch
# from transformers import AutoTokenizer, DistilBertForSequenceClassification

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Function to predict text
def predict_text(text):
    # Tokenize input text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Interpret output
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    return predicted_class




class_names = ["High", "Low", "Medium"]

while True:
    user_input = input("Enter a text: ")
    if user_input.lower() == 'exit':
        break
    prediction = predict_text(user_input)
    predicted_class_name = class_names[prediction]
    print(f"Predicted class: {prediction}: {predicted_class_name}")
# while True:
#     user_input = input("Enter a text: ")
#     if user_input.lower() == 'exit':
#         break
#     prediction = predict_text(user_input)
#     print("Predicted class:", prediction)


Enter a text: Hiking the Inca Trail to Machu Picchu in autumn


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)